In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [94]:
train = pd.read_csv('data/raw/train.csv').set_index('PassengerId').drop(['Cabin', 'Ticket'], axis = 1)
test = pd.read_csv('data/raw/test.csv').set_index('PassengerId').drop(['Cabin', 'Ticket'], axis = 1)

In [100]:
pd.get_dummies(train.drop('Name', axis = 1))

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,0,1,0,0,1
2,1,1,38.0,1,0,71.2833,1,0,1,0,0
3,1,3,26.0,0,0,7.9250,1,0,0,0,1
4,1,1,35.0,1,0,53.1000,1,0,0,0,1
5,0,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,27.0,0,0,13.0000,0,1,0,0,1
888,1,1,19.0,0,0,30.0000,1,0,0,0,1
889,0,3,NaN,1,2,23.4500,1,0,0,0,1


In [95]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [96]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S


In [97]:
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [98]:
round(train.isnull().sum().sort_values(ascending = False) / train.shape[0], 3)

Age         0.199
Embarked    0.002
Fare        0.000
Parch       0.000
SibSp       0.000
Sex         0.000
Name        0.000
Pclass      0.000
Survived    0.000
dtype: float64

# Embarked

In [21]:
train.loc[(train.Sex == 'female') & (train.Pclass == 1) & train.Ticket.str.contains('113'), :].groupby(['Embarked']).size()

Embarked
C     3
S    12
dtype: int64

In [40]:
train.loc[train.Embarked.isnull(), :]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [22]:
train.loc[train.Embarked.isnull(), 'Embarked'] = 'S'

In [23]:
train_embarked = train.groupby(['PassengerId', 'Embarked']).size().unstack(fill_value = 0).drop(['S'], axis = 1).add_prefix("Embarked_")
test_embarked = test.groupby(['PassengerId', 'Embarked']).size().unstack(fill_value = 0).drop(['S'], axis = 1).add_prefix("Embarked_")

# Age

In [ ]:
train.loc[train.Age.isnull(), :]

# Title

In [ ]:
def get_title(name):
    try:
        result = re.findall(r'(Miss|Mr|Mrs|Ms)\b', name)
        return result[0]
    except:
        return 'Other'

In [ ]:
train['Title'] = train.Name.apply(lambda x: get_title(x))
test['Title'] = test.Name.apply(lambda x: get_title(x))

In [ ]:
train_title = train.groupby(['PassengerId', 'Title']).size().unstack(fill_value = 0).drop(['Other'], axis = 1).add_prefix("Title_")
test_title = test.groupby(['PassengerId', 'Title']).size().unstack(fill_value = 0).drop(['Other'], axis = 1).add_prefix("Title_")

# Sex

In [ ]:
train.Sex = [1 if sex == 'male' else 0 for sex in train.Sex]
test.Sex = [1 if sex == 'male' else 0 for sex in test.Sex]

# Combine & Drop

In [ ]:
train = train.drop(['Embarked', 'Name', 'Ticket', 'Cabin', 'cabin', 'Title'], axis = 1).merge(train_embarked, left_index = True, right_index = True).merge(train_cabin, left_index = True, right_index = True).merge(train_title, left_index = True, right_index = True)
train.head()

In [ ]:
train.shape

In [ ]:
test = test.drop(['Embarked', 'Name', 'Ticket', 'Cabin', 'cabin', 'Title'], axis = 1).merge(test_embarked, left_index = True, right_index = True).merge(test_cabin, left_index = True, right_index = True).merge(test_title, left_index = True, right_index = True)
test.head()

In [ ]:
test.shape

# Imputation

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
imputer = KNNImputer(n_neighbors = 5, add_indicator = False).fit(train.iloc[:, 1:])
imputer

In [ ]:
train.iloc[:, 1:] = imputer.transform(train.iloc[:, 1:])
train.head()

In [ ]:
train.isna().sum()

In [ ]:
test[:] = imputer.transform(test)
test.head()

In [ ]:
test.isna().sum()

# Write

In [ ]:
train.to_csv("data/clean/train.csv")
test.to_csv("data/clean/test.csv")